In [1]:
from preprocessing import single_file_processing
from pathlib import Path

In [2]:
file_path_as_object = Path(r"..\data\53980.png")
a = single_file_processing(file_path_as_object)

In [3]:
a

('53980.png', [<PIL.Image.Image image mode=RGB size=224x224>], None)

In [4]:
import os
import concurrent.futures
from pathlib import Path
from tqdm.notebook import tqdm  # 주피터 노트북용 tqdm
from PIL import Image
import time

# --- ⚙️ 설정 (여기를 수정하세요) ---

# 1. 원본 파일이 있는 폴더 경로
INPUT_FOLDER = Path(r"..\data") 

# 2. 전처리된 이미지를 저장할 폴더 경로
OUTPUT_FOLDER = Path("./output_preprocessed")

# 3. 사용할 CPU 코어 수 (None으로 설정 시 가용한 모든 코어 사용)
MAX_WORKERS = None 

# ------------------------------------

# preprocessing.py에서 필요한 함수와 변수 임포트
try:
    from preprocessing import (
        single_file_processing,
        IMAGE_EXTS,
        VIDEO_EXTS,
    )
    print("✅ 'preprocessing.py' 모듈 임포트 성공!")
except ImportError:
    print("❌ 오류: 'preprocessing.py' 파일을 찾을 수 없거나, \n"
          "       파일 내에 'single_file_processing', 'IMAGE_EXTS', 'VIDEO_EXTS' 변수가 없습니다.")
    # 이 경우, 아래 코드가 실행되지 않도록 가짜 함수를 만듭니다.
    def single_file_processing(*args): 
        return "Import Error", None, "Import Error"
    IMAGE_EXTS, VIDEO_EXTS = set(), set()


def get_all_files(folder_path: Path) -> list[Path]:
    """지정된 폴더에서 지원하는 확장자의 모든 파일 목록을 반환합니다."""
    supported_exts = IMAGE_EXTS.union(VIDEO_EXTS)
    file_list = []
    
    print(f"파일 검색 중... (확장자: {supported_exts})")
    
    # 하위 폴더까지 모두 검색 (rglob)
    all_files = list(folder_path.rglob('*'))
    
    file_list = [f for f in all_files if f.suffix.lower() in supported_exts]
    
    return file_list


def run_processing():
    """메인 병렬 처리 실행 함수"""
    
    print(f"📁 입력 폴더: {INPUT_FOLDER}")
    print(f"💾 출력 폴더: {OUTPUT_FOLDER}")

    # 출력 폴더 생성
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

    # 1. 처리할 파일 목록 가져오기
    try:
        file_list = get_all_files(INPUT_FOLDER)
        if not file_list:
            print(f"❌ 경고: '{INPUT_FOLDER}'에서 처리할 파일을 찾을 수 없습니다.")
            return
        print(f"📊 총 {len(file_list)}개의 파일(이미지/비디오)을 찾았습니다.")
        
    except FileNotFoundError:
        print(f"❌ 오류: 입력 폴더 '{INPUT_FOLDER}'를 찾을 수 없습니다.")
        return

    success_count = 0
    fail_count = 0
    start_time = time.time()
    
    # 2. ProcessPoolExecutor를 사용한 병렬 처리
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        
        futures = {
            executor.submit(single_file_processing, file_path): file_path
            for file_path in file_list
        }

        print("🚀 전처리를 시작합니다...")
        
        # tqdm.notebook.tqdm 사용
        pbar = tqdm(
            concurrent.futures.as_completed(futures), 
            total=len(file_list),
            ncols=100
        )
        
        for future in pbar:
            # 3. 완료된 작업 결과 처리
            try:
                filename, image_list, error_msg = future.result()

                if error_msg:
                    # 전처리 중 오류 발생
                    tqdm.write(f"[실패] {filename}: {error_msg}") # tqdm 진행바와 겹치지 않게 출력
                    fail_count += 1
                
                elif image_list and image_list[0] is not None:
                    # 전처리 성공 및 이미지 저장
                    processed_image = image_list[0]
                    
                    output_filename = Path(filename).stem + ".png"
                    save_path = OUTPUT_FOLDER / output_filename
                    
                    processed_image.save(save_path, "PNG")
                    success_count += 1
                
                else:
                    tqdm.write(f"[실패] {filename}: 알 수 없는 오류 (결과 이미지 없음)")
                    fail_count += 1

            except Exception as e:
                original_path = futures[future]
                tqdm.write(f"[치명적 오류] {original_path.name}: {e}")
                fail_count += 1

    end_time = time.time()
    print("\n--- ✨ 처리 완료 ---")
    print(f"✅ 성공: {success_count} 건")
    print(f"❌ 실패: {fail_count} 건")
    print(f"⏱️ 총 소요 시간: {end_time - start_time:.2f} 초")

# --- 코드 실행 ---
# 'preprocessing.py'가 정상적으로 임포트되었을 때만 실행
if 'single_file_processing' in locals() and callable(single_file_processing) and single_file_processing.__name__ != 'Import Error':
    run_processing()
else:
    print("\n[실행 중단] 'preprocessing.py' 임포트 문제로 전처리를 시작할 수 없습니다.")

✅ 'preprocessing.py' 모듈 임포트 성공!
📁 입력 폴더: ..\data
💾 출력 폴더: output_preprocessed
파일 검색 중... (확장자: {'.png', '.jpeg', '.avi', '.jpg', '.mp4'})
📊 총 9개의 파일(이미지/비디오)을 찾았습니다.
🚀 전처리를 시작합니다...


  0%|                                                                         | 0/9 [00:00<?, ?it/s]


--- ✨ 처리 완료 ---
✅ 성공: 9 건
❌ 실패: 0 건
⏱️ 총 소요 시간: 7.35 초


In [5]:
import os
import concurrent.futures
from pathlib import Path
from tqdm.notebook import tqdm  # 주피터 노트북용 tqdm
from PIL import Image
import time
from typing import List, Dict, Any, Union

# --- ⚙️ 설정 (여기를 수정하세요) ---

# 1. 원본 파일이 있는 폴더 경로
INPUT_FOLDER = Path(r"..\data") 

# 2. 사용할 CPU 코어 수 (None으로 설정 시 가용한 모든 코어 사용)
MAX_WORKERS = None 

# (출력 폴더 설정은 필요 없으므로 제거했습니다.)

# ------------------------------------

# preprocessing.py에서 필요한 함수와 변수 임포트
try:
    from preprocessing import (
        single_file_processing,
        IMAGE_EXTS,
        VIDEO_EXTS,
    )
    print("✅ 'preprocessing.py' 모듈 임포트 성공!")
except ImportError:
    print("❌ 오류: 'preprocessing.py' 파일을 찾을 수 없거나, \n"
          "       파일 내에 'single_file_processing', 'IMAGE_EXTS', 'VIDEO_EXTS' 변수가 없습니다.")
    def single_file_processing(*args): 
        return "Import Error", None, "Import Error"
    IMAGE_EXTS, VIDEO_EXTS = set(), set()


def get_all_files(folder_path: Path) -> List[Path]:
    """지정된 폴더에서 지원하는 확장자의 모든 파일 목록을 반환합니다."""
    supported_exts = IMAGE_EXTS.union(VIDEO_EXTS)
    file_list = []
    
    print(f"파일 검색 중... (확장자: {supported_exts})")
    all_files = list(folder_path.rglob('*'))
    file_list = [f for f in all_files if f.suffix.lower() in supported_exts]
    
    return file_list


def run_processing() -> List[Dict[str, Any]]:
    """
    메인 병렬 처리 실행 함수
    
    반환:
        처리 결과 딕셔너리의 리스트.
        예: [{'filename': 'a.jpg', 'status': 'success', 'data': <PIL.Image>},
             {'filename': 'b.mp4', 'status': 'failed', 'data': 'Error: Video has 0 frames'}]
    """
    
    print(f"📁 입력 폴더: {INPUT_FOLDER}")

    # 1. 처리할 파일 목록 가져오기
    try:
        file_list = get_all_files(INPUT_FOLDER)
        if not file_list:
            print(f"❌ 경고: '{INPUT_FOLDER}'에서 처리할 파일을 찾을 수 없습니다.")
            return []
        print(f"📊 총 {len(file_list)}개의 파일(이미지/비디오)을 찾았습니다.")
        
    except FileNotFoundError:
        print(f"❌ 오류: 입력 폴더 '{INPUT_FOLDER}'를 찾을 수 없습니다.")
        return []

    # --- ⭐️ 결과를 저장할 리스트 초기화 ---
    results_list: List[Dict[str, Any]] = []
    
    success_count = 0
    fail_count = 0
    start_time = time.time()
    
    # 2. ProcessPoolExecutor를 사용한 병렬 처리
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        
        futures = {
            executor.submit(single_file_processing, file_path): file_path
            for file_path in file_list
        }

        print("🚀 전처리를 시작합니다...")
        
        pbar = tqdm(
            concurrent.futures.as_completed(futures), 
            total=len(file_list),
            ncols=100
        )
        
        for future in pbar:
            original_path = futures[future]
            filename = original_path.name
            
            # 3. 완료된 작업 결과 처리
            try:
                # future.result()로 (filename, image_list, error) 튜플을 받음
                _filename_from_func, image_list, error_msg = future.result()

                if error_msg:
                    # 전처리 중 오류 발생
                    tqdm.write(f"[실패] {filename}: {error_msg}")
                    fail_count += 1
                    # --- ⭐️ 리스트에 실패 결과 추가 ---
                    results_list.append({
                        'filename': filename,
                        'status': 'failed',
                        'data': error_msg
                    })
                
                elif image_list and image_list[0] is not None:
                    # 전처리 성공
                    processed_image = image_list[0]
                    success_count += 1
                    # --- ⭐️ 리스트에 성공 결과 (이미지 객체) 추가 ---
                    results_list.append({
                        'filename': filename,
                        'status': 'success',
                        'data': processed_image
                    })
                
                else:
                    # 알 수 없는 이유로 이미지가 반환되지 않음
                    tqdm.write(f"[실패] {filename}: 알 수 없는 오류 (결과 이미지 없음)")
                    fail_count += 1
                    results_list.append({
                        'filename': filename,
                        'status': 'failed',
                        'data': 'Unknown error (No image returned)'
                    })

            except Exception as e:
                # 작업 자체에서 예외 발생
                tqdm.write(f"[치명적 오류] {filename}: {e}")
                fail_count += 1
                # --- ⭐️ 리스트에 치명적 오류 결과 추가 ---
                results_list.append({
                    'filename': filename,
                    'status': 'failed',
                    'data': f"Fatal error in process: {e}"
                })

    end_time = time.time()
    print("\n--- ✨ 처리 완료 ---")
    print(f"✅ 성공: {success_count} 건")
    print(f"❌ 실패: {fail_count} 건")
    print(f"⏱️ 총 소요 시간: {end_time - start_time:.2f} 초")
    
    # --- ⭐️ 최종 결과 리스트 반환 ---
    return results_list

# --- 코드 실행 ---
if 'single_file_processing' in locals() and callable(single_file_processing) and single_file_processing.__name__ != 'Import Error':
    # 'results' 변수에 모든 처리 결과 리스트가 저장됩니다.
    results = run_processing()
    
    print("\n--- 📋 실행 결과 요약 ---")
    print(f"✅ 'results' 리스트에 총 {len(results)}개의 결과가 저장되었습니다.")
    
    if results:
        print("\n[첫 번째 결과 예시]")
        print(f"  - 파일명: {results[0].get('filename')}")
        print(f"  - 상 태: {results[0].get('status')}")
        
        # 데이터 타입에 따라 다르게 표시
        data_preview = results[0].get('data')
        if isinstance(data_preview, Image.Image):
            print(f"  - 데이터: <PIL.Image object at {id(data_preview)}>")
        else:
            print(f"  - 데이터: {data_preview}")
else:
    print("\n[실행 중단] 'preprocessing.py' 임포트 문제로 전처리를 시작할 수 없습니다.")

✅ 'preprocessing.py' 모듈 임포트 성공!
📁 입력 폴더: ..\data
파일 검색 중... (확장자: {'.png', '.jpeg', '.avi', '.jpg', '.mp4'})
📊 총 9개의 파일(이미지/비디오)을 찾았습니다.
🚀 전처리를 시작합니다...


  0%|                                                                         | 0/9 [00:00<?, ?it/s]


--- ✨ 처리 완료 ---
✅ 성공: 9 건
❌ 실패: 0 건
⏱️ 총 소요 시간: 7.24 초

--- 📋 실행 결과 요약 ---
✅ 'results' 리스트에 총 9개의 결과가 저장되었습니다.

[첫 번째 결과 예시]
  - 파일명: 53983.png
  - 상 태: success
  - 데이터: <PIL.Image object at 2483742876656>


In [6]:
results

[{'filename': '53983.png',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '53981.png',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '53980.png',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '53982.png',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '53984.png',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '970_973.mp4',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '969_897.mp4',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '968_884.mp4',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>},
 {'filename': '971_564.mp4',
  'status': 'success',
  'data': <PIL.Image.Image image mode=RGB size=224x224>}]